In [1]:
import folium
import pandas as pd
import geopandas as gp
from geopy.distance import geodesic

In [22]:
class Cidades:
    url = dict(
        estados="https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/master/csv/estados.csv",
        municipios="https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/master/csv/municipios.csv",
        municipios_topo="https://raw.githubusercontent.com/betafcc/Brasil-Topojsons/master/min/municipios.topo.json"
    )

    def __init__(self):
        self.dataframe = (
            pd.read_csv(self.url["estados"])
            .rename(columns={"nome": "estado"})
            .merge(
                pd.read_csv(self.url["municipios"]).rename(
                    columns={"nome": "municipio"}
                ),
                on="codigo_uf",
            )
            .set_index("codigo_ibge")
        )
        
        self.geodataframe = gp.read_file(self.url["municipios_topo"])

    def codigo_ibge(self, uf, municipio):
        matches = self.dataframe.loc[lambda df: df.uf == uf].loc[
            lambda df: df.municipio == municipio
        ]

        if len(matches) < 1:
            raise ValueError("No matches")
        elif len(matches) > 1:
            raise ValueError("More than one matched")
        else:
            return matches.iloc[0].name

    def distance(self, uf, municipio):
        cidade = self.dataframe.loc[self.codigo_ibge(uf, municipio)]
        latitude, longitude = cidade.latitude, cidade.longitude

        return self.dataframe.apply(
            lambda r: geodesic(
                (latitude, longitude), (r.latitude, r.longitude)
            ).kilometers,
            axis="columns",
        ).rename("distance")

    def cities_in_radius(self, uf, municipio, radius):
        return self.dataframe.join(
            self.distance(uf, municipio).loc[lambda s: s <= radius], how="right"
        ).sort_values("distance")
    
    def show(self, uf, municipio, radius):
        cidade = self.dataframe.loc[self.codigo_ibge(uf, municipio)]

        df = self.geodataframe[lambda df: df.id.astype(int).isin(
            set(self.cities_in_radius(uf, municipio, radius).index)
        )]
        df.crs = {'init' :'epsg:4326'}
        
        c = df.centroid
        xi, yi, xf, yf = df.geometry.unary_union.bounds

        m = folium.Map()
        folium.Choropleth(geo_data=df, location=[c.y.sum() / len(c), c.x.sum()], fill_color='red').add_to(m)
        folium.Circle(
            location=[cidade.latitude, cidade.longitude],
            radius=radius * 1000,
            color='#3186cc',
            fill=True,
            fill_color='#3186cc'
        ).add_to(m)
        m.fit_bounds([[yi, xi], [yf, xf]])
        return m

In [23]:
cidades = Cidades()

In [24]:
cidades.cities_in_radius(uf='SC', municipio='Florianópolis', radius=30)

,codigo_uf,uf,estado,municipio,latitude,longitude,capital,distance
codigo_ibge,,,,,,,,
4205407,42,SC,Santa Catarina,Florianópolis,-27.5945,-48.5477,1,0.000000
4216602,42,SC,Santa Catarina,São José,-27.6136,-48.6366,0,9.027738
4211900,42,SC,Santa Catarina,Palhoça,-27.6455,-48.6697,0,13.302199
4202305,42,SC,Santa Catarina,Biguaçu,-27.4960,-48.6598,0,15.547768
4201208,42,SC,Santa Catarina,Antônio Carlos,-27.5191,-48.7660,0,23.121993
4215703,42,SC,Santa Catarina,Santo Amaro da Imperatriz,-27.6852,-48.7813,0,25.148988
4217253,42,SC,Santa Catarina,São Pedro de Alcântara,-27.5665,-48.8048,0,25.574999
4200606,42,SC,Santa Catarina,Águas Mornas,-27.6963,-48.8243,0,29.534681


In [27]:
cidades.show(uf='SC', municipio='Florianópolis', radius=50)